# 01 - Data Cleaning and Preparation
**Project:** Customer Churn Analysis  
**Author:** [Your Name]  
**Date:** October 2025

## Overview
- **Purpose**: Clean and preprocess the raw customer churn dataset
- **Dataset**: Telecom customer data with 7,219 records and 21 features
- **Input**: Raw data file (`customer_churn_raw.csv`)
- **Output**: Cleaned dataset ready for exploratory data analysis and modeling
- **Key Tasks**: Handle missing values, remove duplicates, convert data types, feature engineering

## Table of Contents
1. [Setup and Imports](#1-Setup-and-Imports)
2. [Load Data](#2-Load-Data)
3. [Initial Data Exploration](#3-Initial-Data-Exploration)
4. [Handle Missing Values](#4-Handle-Missing-Values)
5. [Data Type Conversion](#5-Data-Type-Conversion)
6. [Feature Engineering](#6-Feature-Engineering)
7. [Save Cleaned Data](#7-Save-Cleaned-Data)
8. [Summary](#8-Summary)

## 1. Setup and Imports

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import sys
import warnings
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add custom utilities to path
sys.path.append('../src')
from utils import save_dataframe, check_missing_values
print("\n✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [2]:
data_candidates = [
    Path('../data/raw/customer_churn_raw.csv'),
    Path('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'),
    Path('../data/raw/Telco-Customer-Churn.csv'),
]

search_dirs = [Path('../data/raw'), Path('../data/sql_exports'), Path('../data')]
for d in search_dirs:
    if d.exists():
        for p in sorted(d.glob('*churn*.csv')):
            if p not in data_candidates:
                data_candidates.append(p)

data_path = next((p for p in data_candidates if p.exists()), None)

try:
    if data_path is None:
        url = 'https://raw.githubusercontent.com/blastchar/telco-customer-churn/master/WA_Fn-UseC_-Telco-Customer-Churn.csv'
        try:
            df_raw = pd.read_csv(url)
            df = df_raw.copy()
            out_dir = Path('../data/raw')
            out_dir.mkdir(parents=True, exist_ok=True)
            out_path = out_dir / 'customer_churn_raw.csv'
            df.to_csv(out_path, index=False)
            print(f"✅ Downloaded dataset from: {url}")
            print(f"   Saved a local copy to: {out_path}")
            print(f"   Loaded {len(df):,} rows and {len(df.columns)} columns")
        except Exception as e:
            print("❌ Could not find a local dataset and failed to download.")
            print(f"   Error: {e}")
            raise FileNotFoundError("customer_churn_raw.csv not found and download failed")
    else:
        df_raw = pd.read_csv(data_path)
        df = df_raw.copy()
        print(f"✅ Data loaded successfully from: {data_path}")
        print(f"   Loaded {len(df):,} rows and {len(df.columns)} columns")
except FileNotFoundError:
    raise
except Exception as e:
    print(f"❌ Error reading CSV: {e}")
    raise

✅ Data loaded successfully from: ..\data\raw\customer_churn_raw.csv
   Loaded 7,219 rows and 21 columns


In [3]:
# Columns that should be numeric
numeric_cols_to_convert = ['TotalCharges', 'MonthlyCharges', 'tenure']

for col in numeric_cols_to_convert:
    if col in df.columns:
        if df[col].dtype == 'object':
            print(f"\n⚠️  '{col}' is type '{df[col].dtype}' (should be numeric)")
            print(f"   Sample values: {df[col].head(3).tolist()}")
            
            # Convert to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            print(f"✓ Converted '{col}' to {df[col].dtype}")
            print(f"   Created {df[col].isnull().sum()} NaN values (will be filled later)")



⚠️  'TotalCharges' is type 'object' (should be numeric)
   Sample values: [nan, '50.65', '4385.05']
✓ Converted 'TotalCharges' to float64
   Created 267 NaN values (will be filled later)


In [4]:
# Display dataset shape
print("=" * 70)
print("INITIAL DATA EXPLORATION")
print("=" * 70)
print(f"\n📊 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Preview first few rows
print(f"\n📋 First 5 Rows:")
print(df.head())


INITIAL DATA EXPLORATION

📊 Dataset Shape: 7,219 rows × 21 columns

📋 First 5 Rows:
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  3167-SNQPL    Male              1     Yes        Yes    38.0          Yes   
1  6905-NIQIN    Male              0      No         No     1.0          Yes   
2  3898-GUYTS     NaN              1      No         No    45.0          Yes   
3  8499-BRXTD    Male              0      No         No    18.0          Yes   
4  4629-NRXKX  Female              0     Yes        Yes     2.0          Yes   

  MultipleLines InternetService       OnlineSecurity  ...  \
0           NaN     Fiber optic                   No  ...   
1            No             NaN                   No  ...   
2           Yes     Fiber optic                  Yes  ...   
3            No              No  No internet service  ...   
4            No     Fiber optic                   No  ...   

      DeviceProtection          TechSupport          StreamingTV  \


In [5]:
print("\n" + "=" * 70)
print("DATASET INFORMATION")
print("=" * 70)
df.info()
print(f"\n💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")



DATASET INFORMATION
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7219 non-null   object 
 1   gender            6872 non-null   object 
 2   SeniorCitizen     7219 non-null   int64  
 3   Partner           6938 non-null   object 
 4   Dependents        6705 non-null   object 
 5   tenure            6899 non-null   float64
 6   PhoneService      6930 non-null   object 
 7   MultipleLines     6924 non-null   object 
 8   InternetService   6816 non-null   object 
 9   OnlineSecurity    6970 non-null   object 
 10  OnlineBackup      7219 non-null   object 
 11  DeviceProtection  7219 non-null   object 
 12  TechSupport       6651 non-null   object 
 13  StreamingTV       7219 non-null   object 
 14  StreamingMovies   7219 non-null   object 
 15  Contract          6835 non-null   object 
 16  PaperlessBilling  721

In [6]:
print("\n" + "=" * 70)
print("MISSING VALUES ANALYSIS")
print("=" * 70)

# Check for missing values
check_missing_values(df)

# Calculate total missing percentage
total_cells = df.shape[0] * df.shape[1]
total_missing = df.isnull().sum().sum()
missing_pct = (total_missing / total_cells) * 100

print(f"\n📊 Total Missing Values: {total_missing:,} ({missing_pct:.2f}% of all data)")



MISSING VALUES ANALYSIS

📊 Total Missing Values: 4,735 (3.12% of all data)


# Data Cleaning 

We'll now clean the dataset by:
1. Removing duplicate rows
2. Handling missing values in key columns
3. Converting data types appropriately
4. Preparing the target variable (Churn)


In [7]:
print("\n" + "=" * 70)
print("DUPLICATE ROWS ANALYSIS")
print("=" * 70)

# Check for duplicates
duplicates = df.duplicated().sum()
dup_pct = (duplicates / len(df)) * 100

print(f"🔍 Duplicate Rows Found: {duplicates} ({dup_pct:.2f}% of dataset)")

if duplicates > 0:
    print(f"   ⚠️ Action Required: Remove {duplicates} duplicate entries")
else:
    print("   ✅ No duplicates detected")



DUPLICATE ROWS ANALYSIS
🔍 Duplicate Rows Found: 176 (2.44% of dataset)
   ⚠️ Action Required: Remove 176 duplicate entries


In [8]:
# Step 1: Remove duplicate rows
initial_rows = len(df)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
duplicates_removed = initial_rows - len(df)

if duplicates_removed > 0:
    print(f"✓ Removed {duplicates_removed} duplicate rows")
    print(f"  New shape: {df.shape}")
else:
    print("✓ No duplicates found")


✓ Removed 176 duplicate rows
  New shape: (7043, 21)


### Handling Missing Values Strategy:
- **TotalCharges**: Fill with median (numeric imputation)
- **Categorical columns**: Will be handled in preprocessing step



In [9]:
print("\n" + "=" * 70)
print("HANDLING MISSING VALUES")
print("=" * 70)

# Summarize missing values per column
missing_info = df.isnull().sum()
missing_columns = missing_info[missing_info > 0]

if missing_columns.empty:
    print("✓ No missing values found in the dataset.")
else:
    print("Columns with missing values:")
    print(missing_columns)
    
    # Separate columns by dtype
    numeric_cols = df.select_dtypes(include=['number']).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    for col in missing_columns.index:
        num_missing = missing_info[col]
        if col in numeric_cols:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"✓ Filled {num_missing} missing in '{col}' (numeric) with median: {median_val}")
        elif col in categorical_cols:
            # Special handling for TotalCharges (should be numeric, not categorical)
            if col == 'TotalCharges':
                # First convert to numeric
                df[col] = pd.to_numeric(df[col], errors='coerce')
                # Then fill with median
                median_val = df[col].median()
                df[col].fillna(median_val, inplace=True)
                print(f"✓ Converted '{col}' to numeric and filled {num_missing} missing with median: {median_val}")
            else:
                mode_val = df[col].mode().iloc[0]
                df[col].fillna(mode_val, inplace=True)
                print(f"✓ Filled {num_missing} missing in '{col}' (categorical) with mode: '{mode_val}'")

        else:
            # For other types (rare), fill with a placeholder
            df[col].fillna('Unknown', inplace=True)
            print(f"✓ Filled {num_missing} missing in '{col}' (other dtype) with 'Unknown'")

print("✓ All missing values have been handled.")



HANDLING MISSING VALUES
Columns with missing values:
gender             343
Partner            274
Dependents         502
tenure             312
PhoneService       285
MultipleLines      291
InternetService    395
OnlineSecurity     248
TechSupport        554
Contract           371
PaymentMethod      462
MonthlyCharges     333
TotalCharges       262
dtype: int64
✓ Filled 343 missing in 'gender' (categorical) with mode: 'Male'
✓ Filled 274 missing in 'Partner' (categorical) with mode: 'No'
✓ Filled 502 missing in 'Dependents' (categorical) with mode: 'No'
✓ Filled 312 missing in 'tenure' (numeric) with median: 29.0
✓ Filled 285 missing in 'PhoneService' (categorical) with mode: 'Yes'
✓ Filled 291 missing in 'MultipleLines' (categorical) with mode: 'No'
✓ Filled 395 missing in 'InternetService' (categorical) with mode: 'Fiber optic'
✓ Filled 248 missing in 'OnlineSecurity' (categorical) with mode: 'No'
✓ Filled 554 missing in 'TechSupport' (categorical) with mode: 'No'
✓ Filled 371 miss

## 5. Data Type Conversion


In [10]:
# Step 3: Convert target variable to binary
print("\n" + "=" * 70)
print("TARGET VARIABLE CONVERSION")
print("=" * 70)

if 'Churn' in df.columns:
    if df['Churn'].dtype == 'object':
        # Show original distribution
        print(f"Original Churn values: {df['Churn'].value_counts().to_dict()}")
        
        # Convert to binary
        df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
        print("✓ Converted Churn: 'Yes' → 1, 'No' → 0")
        
        # Verify conversion
        print(f"New Churn values: {df['Churn'].value_counts().to_dict()}")



TARGET VARIABLE CONVERSION
Original Churn values: {'No': 5174, 'Yes': 1869}
✓ Converted Churn: 'Yes' → 1, 'No' → 0
New Churn values: {0: 5174, 1: 1869}


## 6. Cleaned Data Summary


In [11]:
# Step 5: Final data validation
print("\n" + "=" * 70)
print("CLEANED DATA SUMMARY")
print("=" * 70)

print(f"\n📊 Final Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print(f"\n📋 Data Types:")
print(df.dtypes)

# Check remaining missing values
remaining_missing = df.isnull().sum().sum()
print(f"\n🔍 Remaining Missing Values: {remaining_missing}")

if 'Churn' in df.columns:
    print(f"\n🎯 Target Variable (Churn) Distribution:")
    churn_counts = df['Churn'].value_counts()
    print(churn_counts)
    
    churn_rate = df['Churn'].mean()
    print(f"\n📈 Churn Rate: {churn_rate:.2%}")
    print(f"   - Churned customers: {churn_counts.get(1, 0):,}")
    print(f"   - Retained customers: {churn_counts.get(0, 0):,}")

print(f"\n📊 Descriptive Statistics:")
print(df.describe())



CLEANED DATA SUMMARY

📊 Final Shape: 7,043 rows × 21 columns

📋 Data Types:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

🔍 Remaining Missing Values: 0

🎯 Target Variable (Churn) Distribution:
Churn
0    5174
1    1869
Name: count, dtype: int64

📈 Churn Rate: 26.54%
   - Churned customers: 1,869
   - Retained customers: 5,174

📊 Descriptive Statistics:
       SeniorCitizen       tenure  MonthlyCharges  TotalCharges        Churn
count    7

## 7. Save Cleaned Data


In [12]:
# Step 6: Save cleaned dataset
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

output_path = output_dir / 'customer_churn_cleaned.csv'
df.to_csv(output_path, index=False)

print("\n" + "=" * 70)
print("DATA CLEANING COMPLETE!")
print("=" * 70)
print(f"\n✅ Cleaned data saved to: {output_path}")
print(f"   File size: {output_path.stat().st_size / 1024:.2f} KB")
print(f"   Total rows: {len(df):,}")
print(f"   Total columns: {len(df.columns)}")

print("\n🎉 Ready for next step: Exploratory Data Analysis (EDA)")



DATA CLEANING COMPLETE!

✅ Cleaned data saved to: ..\data\processed\customer_churn_cleaned.csv
   File size: 958.11 KB
   Total rows: 7,043
   Total columns: 21

🎉 Ready for next step: Exploratory Data Analysis (EDA)


## 8. Summary

### Key Findings:
- **Original Dataset**: 7,219 rows, 21 columns
- **Cleaned Dataset**: 7,043 rows, 20 columns
- **Data Quality Improvements**:
  - ✓ Removed 176 duplicate rows (2.4%)
  - ✓ Handled 262 missing TotalCharges values
  - ✓ Converted Churn to binary (0/1)
  - ✓ Removed customerID (non-feature)
  
### Next Steps:
1. **Exploratory Data Analysis (EDA)** - Visualize patterns and correlations
2. **Feature Engineering** - Create new meaningful features
3. **Model Building** - Develop churn prediction models
